In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from numpy.random import seed

import pandas as pd
import numpy as np
import string, os 

In [ ]:
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [ ]:
import requests
import time

import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import re
import glob
import random
import seaborn as sns
import string

from IPython.display import clear_output

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# http://www.nltk.org/howto/wordnet.html

from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.wsd import lesk

In [ ]:
df = pd.read_csv('../input/qa-csv/qa.csv', delimiter='\t', encoding='utf-8', index_col=0)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
train_data = df

In [ ]:
train_data = train_data.rename({'Query': 'sentence'}, axis='columns')

In [ ]:
train_data.head()

In [ ]:
train_data['sent_id'] = np.arange(len(train_data))

In [ ]:
train_data

In [ ]:
train_data['sentence'] = train_data['sentence'].fillna('')

In [ ]:
sentence_tags = []
sentences = train_data['sentence']
for query in sentences:
    print(query)
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    sentence_tags.append(nlp_pos)

In [ ]:
def return_nlp_pos(query):
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    return nlp_pos

In [ ]:
sentence_tags[:5]

In [ ]:
train_data = train_data.rename({'sentence':'Query'}, axis='columns')

In [ ]:
train_data['sentence'] = sentence_tags

In [ ]:
train_data.head()

In [ ]:
train_data['sentence_clean'] = '<s ' + train_data['sentence']
train_data['sentence_clean'] = train_data['sentence_clean'] + ' /s>'

train_data.head()

In [ ]:
# Uni

In [ ]:
text = train_data['sentence_clean']
text_list = " ".join(map(str, text))
text_list[0:100]

In [ ]:
word_list = pd.DataFrame({'words':text.str.split(' ', expand = True).stack().unique()})

In [ ]:
word_list

In [ ]:
word_count_table = pd.DataFrame()
for n,word in enumerate(word_list['words']):
    # Create a list of just the word we are interested in, we use regular expressions so that part of words do not count
    # e.g. 'ear' would be counted in each appearance of the word 'year'
    word_count = len(re.findall(' ' + word + ' ', text_list))  
    word_count_table = word_count_table.append(pd.DataFrame({'count':word_count}, index=[n]))
    
    clear_output(wait=True)
    print('Proportion of words completed:', np.round(n/len(word_list),4)*100,'%')

word_list['count'] = word_count_table['count']
# Remove the count for the start and end of sentence notation so 
# that these do not inflate the other probabilities
word_list['count'] = np.where(word_list['words'] == '<s' , 0,
                     np.where(word_list['words'] == '/s>', 0,
                     word_list['count']))

In [ ]:
word_list

In [ ]:
word_list['prob'] = word_list['count']/sum(word_list['count'])
word_list.head()

In [ ]:
word_list

In [ ]:
unigram_table = pd.DataFrame()

start_time = time.time()
# Loop through each sentence
# REMOVE ROW LIMIT FOR FULL RUN
for index in train_data[:].index:
    data_row = train_data.iloc[index,:]

    sent_probs = pd.DataFrame()
    # Go through each word in the sentence, lookup the probability of the word and 
    # then find the mulitplicitive product of all probabilities in the sentence.
    sentence_cleaned = data_row['sentence_clean'].split(' ')
    sentence_cleaned = [word for word in sentence_cleaned if word not in ('<s', '/s>')]
    for n,word in enumerate(sentence_cleaned):
        prob = float(word_list[word_list['words']==word]['prob'])
        sent_probs = sent_probs.append(pd.DataFrame({'prob': prob}, index = [n]))
    unigram = sent_probs['prob'].prod(axis=0)
    
    # Create a list of unigram calculation for each sentence
    unigram_table = unigram_table.append(pd.DataFrame({'unigram':unigram},index = [index]))
    
    clear_output(wait=True)
    print('Proportion of sentences completed:', np.round(index/len(train_data),4)*100,'%')
        
end_time = time.time()
print('Total run time = ', np.round(end_time-start_time,2)/60, ' minutes')

train_data['unigram'] = unigram_table['unigram']

In [ ]:
train_data

In [ ]:
# Logarithmic Method

In [ ]:
unigram_table_log = pd.DataFrame()

start_time_log = time.time()
# Loop through each sentence
# REMOVE ROW LIMIT FOR FULL RUN
for index in train_data[:].index:
    data_row = train_data.iloc[index,:]

    sent_probs = pd.DataFrame()
    # Go through each word in the sentence, lookup the probability of the word and 
    # then find the mulitplicitive product of all probabilities in the sentence.
    sentence_cleaned = data_row['sentence_clean'].split(' ')
    sentence_cleaned = [word for word in sentence_cleaned if word not in ('<s', '/s>')]
    for n,word in enumerate(sentence_cleaned):
        prob = float(word_list[word_list['words']==word]['prob'])
        log_prob = np.log10(prob)
        sent_probs = sent_probs.append(pd.DataFrame({'log_prob':log_prob}, index = [n]))
        
    unigram_log = sum(sent_probs['log_prob'])
    
    # Create a list of unigram calculation for each sentence
    unigram_table_log = unigram_table_log.append(pd.DataFrame({'unigram_log':unigram_log},index = [index]))
                                         
    clear_output(wait=True)
    print('Proportion of sentences completed:', np.round(index/len(train_data),4)*100,'%')
                                                                   
end_time_log = time.time()
print('Total run time = ', np.round(end_time_log-start_time_log,2)/60, ' minutes')

In [ ]:
unigram_table_log

In [ ]:
unigram_table

In [ ]:
word_1 = 'red'
word_2 = 'dress'

word_1_pos = return_nlp_pos(word_1)
word_2_pos = return_nlp_pos(word_2)
print('1:', word_1_pos, '2:', word_2_pos)

prob_word_1 = word_list[word_list['words'] == word_1_pos]['prob'].iloc[0]
prob_word_2 = word_list[word_list['words'] == word_2_pos]['prob'].iloc[0]

unigram_prob = prob_word_1*prob_word_2

print('The unigram probability of the word "red" occuring given the word "dress" was the previous word is: ', np.round(unigram_prob,10))